# MODEL TRAINING 
## using 5 cross validation

In [1]:
import joblib
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

In [2]:
tr_feature = pd.read_csv('train_features.csv')
tr_label = pd.read_csv('train_label.csv')
vl_feature = pd.read_csv('val_features.csv')
vl_label = pd.read_csv('val_label.csv')
ts_feature = pd.read_csv('test_features.csv')
ts_label = pd.read_csv('test_label.csv')

In [9]:
def print_result(results):
    print(f'BEST PARAMS :{results.best_params_}')
    mean = results.cv_results_['mean_test_score']
    stds = results.cv_results_['std_test_score']
    params = results.cv_results_['params']
    
    for mean,std,param in zip(mean,stds,params):
        print(f'{round(mean,3)} +/-{round(std,3)} for{param}')

In [22]:
svm_model = SVC()
parameters = {
    'C' :[0.001,0.01,0.1,1,5,10],
    'kernel' :['linear','rbf']
}
cv = GridSearchCV(svm_model,parameters,cv=5)
cv.fit(tr_feature,tr_label.values.ravel())


GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 5, 10],
                         'kernel': ['linear', 'rbf']})

In [25]:
print_result(cv)

BEST PARAMS :{'C': 10, 'kernel': 'linear'}
0.78 +/-0.0 for{'C': 0.001, 'kernel': 'linear'}
0.78 +/-0.0 for{'C': 0.001, 'kernel': 'rbf'}
0.78 +/-0.0 for{'C': 0.01, 'kernel': 'linear'}
0.78 +/-0.0 for{'C': 0.01, 'kernel': 'rbf'}
0.78 +/-0.0 for{'C': 0.1, 'kernel': 'linear'}
0.78 +/-0.0 for{'C': 0.1, 'kernel': 'rbf'}
0.78 +/-0.0 for{'C': 1, 'kernel': 'linear'}
0.78 +/-0.0 for{'C': 1, 'kernel': 'rbf'}
0.78 +/-0.0 for{'C': 5, 'kernel': 'linear'}
0.78 +/-0.0 for{'C': 5, 'kernel': 'rbf'}
0.781 +/-0.001 for{'C': 10, 'kernel': 'linear'}
0.78 +/-0.0 for{'C': 10, 'kernel': 'rbf'}


In [26]:
joblib.dump(cv.best_estimator_,'SVM_MDL.pkl')

['SVM_MDL.pkl']

In [27]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
from time import time

In [28]:
model = {'SVM_MDL.pkl':joblib.load('SVM_MDL.pkl')}

In [29]:
def evaluuate_model(name,model,features,labels):
    start = time()
    pred = model.predict(features)
    end = time()
    accuracy = round(accuracy_score(labels,pred),3)
    precision = round(precision_score(labels,pred),3)
    recall = round(recall_score(labels,pred),3)
    print(f'{name}--Accuracy: {accuracy}/ Precision: {precision} / Recall: {recall} / Latency: {round((end-start)*1000,1)}ms')

### Geting accuracy for validation data

In [30]:
for name,mdl in model.items():
    evaluuate_model(name,mdl,vl_feature,vl_label)

SVM_MDL.pkl--Accuracy: 0.787/ Precision: 1.0 / Recall: 0.023 / Latency: 63.6ms


### Geting accuracy for Test data

In [31]:
evaluuate_model('SVM',model['SVM_MDL.pkl'],ts_feature,ts_label)

SVM--Accuracy: 0.798/ Precision: 1.0 / Recall: 0.01 / Latency: 83.6ms
